# Анализ и подготовка данных Sentinel-2

1. Выяснить, данные для каких территорий, за какое время, в каких CRS имеются в наличии. Сформировать таблицу со столбцами:
    - имя директории снимка,
    - дата съемки и время съемки,
    - CRS,
    - количество фрагментов,
    - координаты верхнего левого угла каждого фрагмента,
    - координаты верхнего левого угла верхнего левого фрагмента.
2. Сформировать RGB-композиты, посмотреть на данные.
3. Перевести данные 10-метровых каналов в табличный вид.
4. Вывести гистограммы для каждого снимка, для каждого канала.
5. Нормализовать данные.
6. Объединить данные снимков, состоящих из нескольких фрагментов, обрезать по территории, для которой имеется наибольшее количество данных.

## Imports & configs

In [221]:
# Imports

import rasterio as rio
import numpy as np
import pandas as pd
from datetime import datetime
import re

# Importing and reloading custom modules

import os, sys
from importlib import reload

BASE_DIR = os.path.dirname(os.getcwd())
if BASE_DIR not in sys.path:
    sys.path.append(BASE_DIR)

from src import config, utils

custom_modules = [
    module
    for module_name, module in sys.modules.items()
    if module_name.startswith("src.")
]

for custom_module in custom_modules:
    reload(custom_module)

In [239]:
# Config

experiment_name = "Sentinel-2-data-analysis-and-preparation"
interim_dir, processed_dir = utils.make_experiment_dirs(experiment_name, models=False)

sentinel_data_info_path = os.path.join(interim_dir, "Sentinel-2_data_info.csv")
rgb_dir = os.path.join(processed_dir, "RGB")

## 1. Территории и время съемки

### Подготовка таблицы

In [191]:
raw_data_listdir = os.listdir(config.RAW_DATA_DIR)
sentinel_data_dirnames = [name for name in raw_data_listdir if name.startswith("SENTINEL-2")]
sentinel_data_paths = [os.path.join(config.RAW_DATA_DIR, dirname) for dirname in sentinel_data_dirnames]

band2_paths = []
fragment_counts = []
for data_path in sentinel_data_paths:
    files_list = [filename for filename in os.listdir(data_path) if filename.endswith('tif')]
    band2_filenames = [name for name in files_list if "channel2_" in name]
    fragment_counts.append(len(band2_filenames))
    band2_filename = band2_filenames[0]
    band2_paths.append(os.path.join(data_path, band2_filename))

datetimes = []
crss = []
for path in band2_paths:
    datetime_str = re.search('MSI_(.*)_channel', os.path.basename(path)).group(1)
    datetime_obj = datetime.strptime(datetime_str, '%Y%m%d_%H%M%S')
    datetimes.append(datetime_obj.strftime('%Y-%m-%d %H:%M:%S'))

    with rio.open(path) as src_img:
        crss.append(src_img.crs.to_string())

In [192]:
info = pd.DataFrame(
    {
        "dirname": sentinel_data_dirnames,
        "datetime": datetimes,
        "crs": crss,
        "fragment_count": fragment_counts
    }
)

for i in range(1, info["fragment_count"].max() + 1):
    info[f"fragment_{i}_left_x"] = np.NaN
    info[f"fragment_{i}_top_y"] = np.NaN

info["left_x"] = np.NaN
info["top_y"] = np.NaN

In [193]:
for idx, data_path in enumerate(sentinel_data_paths):
    files_list = [filename for filename in os.listdir(data_path) if filename.endswith('tif')]
    band2_filenames = [name for name in files_list if "channel2_" in name]
    for i, filename in enumerate(band2_filenames):
        fragment_path = os.path.join(data_path, filename)
        with rio.open(fragment_path) as src_img:
            left_x, top_y = src_img.transform * (0, 0)
            info.at[idx, f"fragment_{i + 1}_left_x"] = left_x
            info.at[idx, f"fragment_{i + 1}_top_y"] = top_y
    
    info.at[idx, "left_x"] = info.loc[idx].filter(regex="left_x").min()
    info.at[idx, "top_y"] = info.loc[idx].filter(regex="top_y").max()

In [194]:
info.to_csv(sentinel_data_info_path)

In [195]:
info = pd.read_csv(sentinel_data_info_path, index_col=0)
info

,dirname,datetime,crs,fragment_count,fragment_1_left_x,fragment_1_top_y,fragment_2_left_x,fragment_2_top_y,fragment_3_left_x,fragment_3_top_y,fragment_4_left_x,fragment_4_top_y,left_x,top_y
0,SENTINEL-2A_MSI_20200901_053433,2020-09-01 05:34:33,EPSG:32645,4,300000.0,5800020.0,399960.0,5800020.0,399960.0,5700000.0,300000.0,5700000.0,300000.0,5800020.0
1,SENTINEL-2A_MSI_20200921_053434,2020-09-21 05:34:34,EPSG:32645,4,399960.0,5700000.0,399960.0,5800020.0,300000.0,5800020.0,300000.0,5700000.0,300000.0,5800020.0
2,SENTINEL-2A_MSI_20201117_084357,2020-11-17 08:43:57,EPSG:32637,1,399960.0,6300000.0,NaN,NaN,NaN,NaN,NaN,NaN,399960.0,6300000.0
3,SENTINEL-2A_MSI_20210327_052418,2021-03-27 05:24:18,EPSG:32645,4,300000.0,5700000.0,300000.0,5800020.0,399960.0,5700000.0,399960.0,5800020.0,300000.0,5800020.0
4,SENTINEL-2A_MSI_20220524_053423,2022-05-24 05:34:23,EPSG:32645,4,300000.0,5800020.0,300000.0,5700000.0,399960.0,5800020.0,399960.0,5700000.0,300000.0,5800020.0
5,SENTINEL-2B_MSI_20190425_085420,2019-04-25 08:54:20,EPSG:32637,1,399960.0,6300000.0,NaN,NaN,NaN,NaN,NaN,NaN,399960.0,6300000.0
6,SENTINEL-2B_MSI_20190813_053920,2019-08-13 05:39:20,EPSG:32645,4,399960.0,5700000.0,300000.0,5700000.0,300000.0,5800020.0,399960.0,5800020.0,300000.0,5800020.0
7,SENTINEL-2B_MSI_20210325_085404,2021-03-25 08:54:04,EPSG:32637,1,399960.0,6300000.0,NaN,NaN,NaN,NaN,NaN,NaN,399960.0,6300000.0
8,SENTINEL-2B_MSI_20210511_084252,2021-05-11 08:42:52,EPSG:32637,1,399960.0,6300000.0,NaN,NaN,NaN,NaN,NaN,NaN,399960.0,6300000.0
9,SENTINEL-2B_MSI_20210531_052344,2021-05-31 05:23:44,EPSG:32645,4,300000.0,5700000.0,399960.0,5700000.0,300000.0,5800020.0,399960.0,5800020.0,300000.0,5800020.0


### Информация

Количество снимков Sentinel-2

In [196]:
n_images = len(info)
n_images

12

Координаты левых верхних точек территорий, для которых имеются снимки

In [197]:
territories = np.unique(info[["left_x", "top_y"]], axis=0)
for x, y in territories:
    print(f"({x}, {y})")

(300000.0, 5800020.0)
(399960.0, 6300000.0)


Даты, за которые имеются снимки для каждой территории

In [202]:
for x, y in territories:
    selection = info[(info["left_x"] == x) & (info["top_y"] == y)]
    crs = np.unique(selection["crs"])
    fragment_count = np.unique(selection["fragment_count"])
    print(f"Координаты: ({x}, {y}), CRS: {crs}, количество фрагментов: {fragment_count}, {len(selection)} снимков:")
    for dt in selection["datetime"].sort_values():
        print(f"\t{dt}")

Координаты: (300000.0, 5800020.0), CRS: ['EPSG:32645'], количество фрагментов: [4], 7 снимков:
	2019-08-13 05:39:20
	2020-09-01 05:34:33
	2020-09-21 05:34:34
	2021-03-27 05:24:18
	2021-05-31 05:23:44
	2021-07-03 05:34:05
	2022-05-24 05:34:23
Координаты: (399960.0, 6300000.0), CRS: ['EPSG:32637'], количество фрагментов: [1], 5 снимков:
	2019-04-25 08:54:20
	2020-11-17 08:43:57
	2021-03-25 08:54:04
	2021-05-11 08:42:52
	2021-10-08 08:44:14


## 2. Формирование RGB-композитов

In [236]:
%%time

data_dirs = list(info["dirname"])
for data_dir in data_dirs:
    data_path = os.path.join(config.RAW_DATA_DIR, data_dir)
    fragment_count = int(info[info["dirname"] == data_dir]["fragment_count"])
    bands = [4, 3, 2]  # Red, Green, Blue
    for fragment in range(1, fragment_count + 1):
        bands_paths = [os.path.join(data_path, f"{data_dir}_channel{band}_{fragment}.tif") for band in bands]
        rgb_path = os.path.join(rgb_dir, f"{data_dir}_{fragment}_RGB.tif")
        utils.combine_bands(bands_paths, rgb_path)

CPU times: total: 2min 10s
Wall time: 5min 21s


## 3.